In [ ]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [ ]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [3]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip
# db_ip = localhost # 집에서 본인의 개인 컴퓨터에 설치된 오라클에 접속할때,
# db_ip = '127.0.0.1:1521/xe' # 그러내 컴퓨터에 설치된 DB 로 할때, 학원서버로 실행할때는 주석처리하면 됨!!

In [4]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [5]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [6]:
# _, _, _, table_col_set_ = extract_table_columns_set(file_list[0]) # _, :: 임시로 쓰고 버리는 값.

In [7]:
def extract_table_columns_set(file_nm):
    for key, value in table_nm_set.items():
        if value in file_nm:
            table_nm = key
            table_col_set = col2dict[table_nm]
            
            col_nms = []
            col_dtypes = []
            
            for col in table_col_set:
                col_nms.append(col[1][0])
                col_dtypes.append(col[1][1])

#             col_nms, col_dtypes = col[1][0], col[1][1] for col in table_col_set
            
            return [table_nm , col_nms, col_dtypes, table_col_set]

In [8]:
#######################################
# Test : sqlalchemy
#######################################

# import cx_Oracle
# dsn = cx_Oracle.makedsn('200.1.220.251', 1521, sid='xe')
# print(dsn)

# from sqlalchemy import create_engine

# engine = create_engine("oracle+cx_oracle://"+db_id+":"+db_pwd+"@"+db_ip)

In [9]:
def create_table_stmt(table_nm):
    import re
    
    sql = ''
    
    for idx, col in enumerate(col2dict[table_nm]):
        if( idx != 0 ):
            sql += ' ,'
        sql += col[1][0] + ' ' + col[1][1]    
    
    stmt = ('CREATE TABLE {0}({1})'.format(table_nm, sql)).upper()
    stmt = re.sub('VARCHAR2\((\d?\d)\)',
                  'VARCHAR2(\g<1> CHAR)',
                  stmt)
    return stmt

In [10]:
def create_insert_stmt(table_nm, columns):
    questions = ''
    for idx, col in enumerate(columns):
        if idx:
            questions += ', :{0}'.format(col)
        else:
            questions += ':{0}'.format(col)

    sql = ''
    sql = 'insert into {table_nm} values({questions})'.format(table_nm=table_nm,
                                                                         questions = questions)
    
    return sql

In [11]:
def df_to_oracle(file_nm):
    import pandas as pd
    
    table_columns = extract_table_columns_set(file_nm)
    table_nm = table_columns[0]
    columns = table_columns[1]
    dtypes = table_columns[2]
   
    df = pd.read_csv(origin_path+file_nm, encoding='euc-kr')
    df.fillna(0, inplace=True)
    data = df.values.tolist()

    del df
    
    stmt = create_table_stmt(table_nm)
    sql = create_insert_stmt(table_nm, columns)


    import cx_Oracle
    
    try:
        # establish a new connection
        with cx_Oracle.connect(db_id, db_pwd, db_ip, encoding="UTF-8") as connection:
            # create a cursor
            with connection.cursor() as cursor:
                # execute the create statement
                cursor.execute(stmt)

                # execute the insert statement                
                cursor.execute('DELETE FROM ' + table_nm)
                cursor.executemany(sql, data)
                                                
                # execute the 'DROP TABLE' statement
#                 cursor.execute('DROP TABLE ' + table_nm)

                # commit work
                connection.commit()
    except cx_Oracle.Error as error:
        print('Error occurred:')
        print(error)
        
    print('done')

In [12]:
# 각 엑셀 파일 컬럼 영문으로 변경하면서 db 에 넣기
for file_nm in file_list:
    print(file_nm)
    df_to_oracle(file_nm)
# df_to_oracle('02.상권분석서비스(상권-집객시설).csv')

00.상권분석서비스(상권-영역).csv
Error occurred:
ORA-00955: name is already used by an existing object
done
01.상권분석서비스(상권-추정유동인구).csv
Error occurred:
ORA-00955: name is already used by an existing object
done
02.상권분석서비스(상권-집객시설).csv
Error occurred:
ORA-00955: name is already used by an existing object
done
03.상권분석서비스(상권배후지-추정유동인구).csv
Error occurred:
ORA-00955: name is already used by an existing object
done
04.상권분석서비스(상권배후지-집객시설).csv
Error occurred:
ORA-00955: name is already used by an existing object
done
05.상권분석서비스(상권배후지-상주인구).csv
Error occurred:
ORA-00955: name is already used by an existing object
done
06.상권분석서비스(상권_상주인구).csv
Error occurred:
ORA-00955: name is already used by an existing object
done
07.상권분석서비스(상권배후지-아파트).csv
Error occurred:
ORA-00955: name is already used by an existing object
done
08.상권분석서비스(상권배후지-점포).csv
Error occurred:
ORA-00955: name is already used by an existing object
done
09.상권분석서비스(상권-상권변화지표).csv
Error occurred:
ORA-00955: name is already used by an existing object